In [35]:
from agents import trace,Agent,Runner,function_tool, GuardrailFunctionOutput, InputGuardrail, OutputGuardrail, InputGuardrailTripwireTriggered, OutputGuardrailTripwireTriggered
import os
from dotenv import load_dotenv
from pydantic import BaseModel
from serpapi import GoogleSearch
import gradio as gr

In [36]:
load_dotenv(override=True)
inputinst = "your a agent, who will check if the research is about some person"

class inputformat(BaseModel):
    input:str
    "It just going to have user prompt"
    is_valid:bool
    "it is going to have data. wheather input is valid or not"

input_validation = Agent(name="InputValidation",
                          instructions=inputinst, 
                          model="gpt-4.1-mini",
                          output_type=inputformat)

@InputGuardrail
async def inputValidation(cxt, agent, input):
    response = await Runner.run(starting_agent=input_validation,context=cxt.context, input=input)
    return GuardrailFunctionOutput(output_info=response.final_output, tripwire_triggered=response.final_output.is_valid)

In [ ]:
@function_tool
def webSearch(serchterm: str):
    "it going to do the web search"
    key = os.getenv("SERPAPI_KEY")
    params = {
        "q": f"{serchterm}",
        "location": "Austin, Texas, United States",
        "hl": "en",
        "gl": "us",
        "google_domain": "google.com",
        "api_key": key
            }
    search = GoogleSearch(params)
    results = search.get_dict()
    return results

querygenerator = Agent(name="Querygenerator",
                       instructions="your a agent who is going to generate 1 the search term",
                       model="gpt-4.1-mini", 
                       input_guardrails=[inputValidation])

querygeneratorTool = querygenerator.as_tool(tool_name="querygeneratorTool",
                                            tool_description="going to generate 1 the search term")

researcher = Agent(name="researcher", 
                   instructions="your are researcher. who are going to do the research for input prompt by generating the 1 search terms. Do the web search and Consolidate the output",
                   model="gpt-4.1-mini", 
                   tools=[querygeneratorTool,webSearch],
                   input_guardrails=[inputValidation])

async def run(query:str):
    with trace("Researcher"):
        try:
            response = await Runner.run(researcher,query)
            return response.final_output
        except InputGuardrailTripwireTriggered:
            return "Input Guardrail Triggered you have asked personal information"

In [45]:
with gr.Blocks(theme=gr.themes.Default(primary_hue="amber")) as ui:
    gr.Markdown("# Deep Research")
    query_textbox = gr.Textbox(label="What topic would you like to research?")
    run_button = gr.Button("Run", variant="primary")
    report = gr.Markdown(label="Report")
    
    run_button.click(fn=run, inputs=query_textbox, outputs=report, show_progress='full')
    query_textbox.submit(fn=run, inputs=query_textbox, outputs=report, show_progress='full')

ui.launch(inbrowser=True)

* Running on local URL:  http://127.0.0.1:7863
* To create a public link, set `share=True` in `launch()`.
